In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os,sys
import PIL
import tensorflow as tf
import cv2 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from PIL import Image
from skimage.io import imread, imshow
from skimage.color import rgb2hsv
from matplotlib import pyplot as plt
from PIL import Image # importing image module 
import numpy as np 
import imageio
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image




In [2]:
full_dir = "E:/User Document/processed3"
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  full_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  full_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
class_names = train_ds.class_names
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 841 files belonging to 6 classes.
Using 757 files for training.
Found 841 files belonging to 6 classes.
Using 84 files for validation.
['NitrogenHigh', 'NitrogenLow', 'PhosphorusHigh', 'PhosphorusLow', 'PotassiumHigh', 'PotassiumLow']


In [3]:


num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

print(model)

In [4]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        1

In [5]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
24/24 [==============================] - 15s 575ms/step - loss: 1.4715 - accuracy: 0.3554 - val_loss: 1.3179 - val_accuracy: 0.3333
Epoch 2/10
24/24 [==============================] - 14s 565ms/step - loss: 1.1826 - accuracy: 0.4676 - val_loss: 1.2831 - val_accuracy: 0.4048
Epoch 3/10
24/24 [==============================] - 13s 561ms/step - loss: 1.0813 - accuracy: 0.5099 - val_loss: 1.1307 - val_accuracy: 0.4762
Epoch 4/10
24/24 [==============================] - 14s 571ms/step - loss: 0.9946 - accuracy: 0.5852 - val_loss: 1.2310 - val_accuracy: 0.4881
Epoch 5/10
24/24 [==============================] - 14s 566ms/step - loss: 0.9438 - accuracy: 0.5971 - val_loss: 1.1958 - val_accuracy: 0.4762
Epoch 6/10
24/24 [==============================] - 13s 545ms/step - loss: 0.8726 - accuracy: 0.6473 - val_loss: 0.9836 - val_accuracy: 0.6190
Epoch 7/10
24/24 [==============================] - 13s 559ms/step - loss: 0.7964 - accuracy: 0.6513 - val_loss: 0.9467 - val_accuracy: 0.6071

In [6]:
from PIL import Image
def load_image(img_path, show=False):

    img = Image.open(img_path)
    #img = img1.Rescaling(1./255, input_shape=(img_height, img_width, 3))
    img_tensor = np.array(img)                    #       # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 1./255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor



In [7]:
from PIL import Image
#E:/User Document/processed2/NitrogenLow/NitrogenLow3.jpg
#E:/User Document/processed2/PotassiumLow/164.jpg
#E:/User Document/processed2/PhosphorusHigh/19.jpg
image = Image.open('E:/User Document/processed2/PotassiumLow/164.jpg')
image = image.resize((180, 180))
image = np.array(image)
img_tensor = np.expand_dims(image, axis=0)    
model.predict(img_tensor)

array([[-4.6322722 , -0.18940586, -0.1327353 ,  1.494361  ,  0.3768516 ,
         3.8904536 ]], dtype=float32)

In [8]:
model.save('E:/User Document/my_model5.h5') 